### First run the hyperparameter selection and benchmark

From under the directory:
```
tphenotype/notebooks/benchmark
```
Execute the `bash` script:
```shell
bash run_experiment.sh
```

### Examine benchmark results

In [5]:
import pandas as pd
from IPython.display import display
from IPython.core.display import HTML
from analysis import find_best_method
from benchmarks import prepare_benchmark

In [6]:
metrics = ["ROC", "PRC", "Silhouette_auc", "Hroc", "Hprc"]
synth_metrics = ["ROC", "PRC", "PURITY", "RAND", "MI", "Silhouette_auc", "Hroc", "Hprc"]

result_dir = "benchmark_results"

- KM-Laplace is the KM-$\mathcal{L}$ baseline in the paper
- T-Phenotype (joint) is the ablation T-phenotype (J)
- MI = NMI (normalized mutual information)
- Silhouette_auc = AUCSIL in the paper


In [7]:
table_order = [
    "KM-E2P(y)",
    "KM-E2P(z)",
    "KM-DTW-D",
    "KM-Laplace",
    "SEQ2SEQ",
    "AC-TPC",
    "T-Phenotype(joint)",
    "T-Phenotype",
]

In [ ]:
for dataname in ["Synth", "ICU", "ADNI"]:
    results = pd.read_csv(f"{result_dir}/{dataname}_benchmark_complete.csv", index_col=0)
    results = results.reset_index(drop=True)
    r = results.set_index("method")
    if dataname == "Synth":
        r = r.loc[table_order + ["Spectral-DTW-D"]]  # KM + spectral clustering is possible on the synthetic dataset
    else:
        r = r.loc[table_order]
    if dataname == "Synth":
        m = synth_metrics
    else:
        m = metrics
    ttest = find_best_method(r.reset_index(), m)
    print(f"dataset: {dataname}")
    print("performance")
    display(HTML(r[m].to_html()))
    # p-value of two-sample test (of equal mean)
    print("p-value")
    display(HTML(ttest.to_html()))

### Check hyperparameters

In [ ]:
def print_hparams(model, config, loss_weights):
    if model.__name__ == "KME2P":
        hparams = ["num_layers", "hidden_size", "latent_size"]
        print(f'model: KME2P{config["latent_space"]}')
        for k in hparams:
            print(f"{k}={config[k]}")
        print()
    elif model.__name__ == "Predictor":
        print(f"model: T-Phenotype")
        print("encoder parameters:")
        for k in ["pole_separation", "max_degree"]:
            print(f'{k}={config["encoder_config"][k]}')
        for k in ["pole", "real"]:
            print(f"loss coeff {k}={loss_weights[k]}")

        print("predictor parameters:")
        for k in ["num_layer", "hidden_size"]:
            print(f"{k}={config[k]}")
        print()


for dataname in ["Synth", "ICU", "ADNI"]:
    splits, setup_list = prepare_benchmark(dataname)
    print("hyperparameters")

    for model, config, loss_weights in setup_list:
        print_hparams(model, config, loss_weights)
    print()

Hyperparameters of T-Phenotype

Laplace encoder
- pole = $\alpha$
- real = $\alpha_1$
- pole_separation = $\delta_{pole}$
- max_degree = $d$